In [1]:
import random
import time
import itertools
from collections import defaultdict
import math

In [2]:
# ---------------------------
# k-SAT instance generator
# ---------------------------
def generate_k_sat(n, m, k, seed=None):
    """Generate uniform random k-SAT instance with n variables, m clauses, clause length k."""
    if seed is not None:
        random.seed(seed)
    clauses = set()
    vars_range = list(range(1, n+1))
    while len(clauses) < m:
        # choose k distinct variables
        vars_sel = random.sample(vars_range, k)
        # randomly flip sign
        lits = []
        for v in vars_sel:
            sign = random.choice([1, -1])
            lits.append(sign * v)
        # ensure clause has distinct literals (by variable) - already ensured
        clause = tuple(lits)
        # avoid duplicate clause (order-insensitive)
        # canonicalize by sorting absolute var then sign to avoid duplicates
        canon = tuple(sorted(clause, key=lambda x: (abs(x), x)))
        clauses.add(canon)
    # return list of clauses (each is tuple of ints)
    return [tuple(c) for c in clauses]


In [3]:
# ---------------------------
# Evaluation helpers
# ---------------------------
def evaluate_clause(clause, assignment):
    """Return True if clause satisfied under assignment (assignment is dict or list indexed 1..n)."""
    for lit in clause:
        var = abs(lit)
        val = assignment[var]
        if lit > 0 and val:
            return True
        if lit < 0 and not val:
            return True
    return False

def unsatisfied_count(clauses, assignment):
    return sum(1 for c in clauses if not evaluate_clause(c, assignment))

def clause_true_literals(clause, assignment):
    count = 0
    for lit in clause:
        var = abs(lit)
        val = assignment[var]
        if (lit > 0 and val) or (lit < 0 and not val):
            count += 1
    return count


In [4]:
# ---------------------------
# Heuristics
# ---------------------------
def h1(clauses, assignment):
    """Heuristic 1: number of unsatisfied clauses."""
    return unsatisfied_count(clauses, assignment)

def h2(clauses, assignment):
    """Heuristic 2: unsatisfied + small bias by 'distance' to satisfaction."""
    unsat = 0
    total_distance = 0
    for c in clauses:
        true_lits = clause_true_literals(c, assignment)
        if true_lits == 0:
            unsat += 1
            total_distance += (len(c) - true_lits)
    return unsat + 0.01 * total_distance


In [5]:
# ---------------------------
# Neighbor generation
# ---------------------------
def flip_var(assignment, var_index):
    new = assignment.copy()
    new[var_index] = not new[var_index]
    return new

def random_pair(n):
    a = random.randint(1, n)
    b = random.randint(1, n)
    while b == a:
        b = random.randint(1, n)
    return a, b


In [6]:
# ---------------------------
# Hill Climbing (best-improvement single flip)
# ---------------------------
def hill_climbing(clauses, n, heuristic_fn, max_iters=1000, seed=None):
    if seed is not None:
        random.seed(seed)
    # random initial assignment [index 0 unused]
    assignment = [None] + [random.choice([False, True]) for _ in range(n)]
    g = heuristic_fn(clauses, assignment)  # current cost
    history = []
    for step in range(max_iters):
        if g == 0:
            return True, step, assignment, history
        # evaluate all single-flip neighbors and pick best
        best = None
        best_cost = math.inf
        best_assign = None
        for var in range(1, n+1):
            neigh = assignment.copy()
            neigh[var] = not neigh[var]
            cost = heuristic_fn(clauses, neigh)
            if cost < best_cost:
                best_cost = cost
                best_assign = neigh
                best = var
        if best_cost < g:
            assignment = best_assign
            g = best_cost
            history.append((step+1, g))
        else:
            # local optimum
            break
    return (g == 0), len(history), assignment, history


In [7]:
# ---------------------------
# Beam Search (single-flip expansions)
# ---------------------------
def beam_search(clauses, n, beam_width=3, heuristic_fn=h1, max_levels=100):
    # beam is list of (assignment, cost)
    def rand_assign():
        return [None] + [random.choice([False, True]) for _ in range(n)]
    beam = []
    # initialize with random beams
    for _ in range(beam_width):
        a = rand_assign()
        beam.append((a, heuristic_fn(clauses, a)))
    history = []
    for level in range(max_levels):
        # check goal
        for a, cost in beam:
            if cost == 0:
                return True, level, a, history
        # expand all neighbors (single-flip)
        candidates = []
        for a, _ in beam:
            for var in range(1, n+1):
                neigh = a.copy()
                neigh[var] = not neigh[var]
                cost = heuristic_fn(clauses, neigh)
                candidates.append((neigh, cost))
        # deduplicate by tuple assignment
        unique = {}
        for a,c in candidates:
            key = tuple(a[1:])
            if key not in unique or c < unique[key][0]:
                unique[key] = (c, a)
        # pick top beam_width by cost (lower better)
        sorted_cands = sorted(unique.values(), key=lambda x: x[0])
        beam = [(a, c) for (c, a) in sorted_cands[:beam_width]]
        history.append((level+1, [c for c,a in beam]))
    # final check
    for a, cost in beam:
        if cost == 0:
            return True, max_levels, a, history
    return False, max_levels, beam[0][0], history



In [8]:
# ---------------------------
# Variable Neighborhood Descent (VND)
# neighborhoods: k=1,2,3 flips
# ---------------------------
def vnd(clauses, n, heuristic_fn, max_no_improve_iter=100, seed=None):
    if seed is not None:
        random.seed(seed)
    assignment = [None] + [random.choice([False, True]) for _ in range(n)]
    current_cost = heuristic_fn(clauses, assignment)
    history = []
    neighborhoods = [1, 2, 3]
    iter_total = 0
    k_index = 0
    while k_index < len(neighborhoods) and iter_total < max_no_improve_iter:
        k = neighborhoods[k_index]
        improved = False
        # generate candidate flips for neighborhood size k
        best_assign = None
        best_cost = current_cost
        # For k small we can try many combos, but to keep time bounded try random subset of combos
        combos_to_try = []
        if k == 1:
            combos_to_try = [(i,) for i in range(1, n+1)]
        else:
            # try up to 200 random combos
            tries = min(200, math.comb(n, k) if n >= k else 0)
            seen = set()
            while len(combos_to_try) < tries:
                picks = tuple(sorted(random.sample(range(1, n+1), k)))
                if picks not in seen:
                    seen.add(picks)
                    combos_to_try.append(picks)
        for combo in combos_to_try:
            neigh = assignment.copy()
            for v in combo:
                neigh[v] = not neigh[v]
            cost = heuristic_fn(clauses, neigh)
            iter_total += 1
            if cost < best_cost:
                best_cost = cost
                best_assign = neigh
        if best_assign is not None and best_cost < current_cost:
            assignment = best_assign
            current_cost = best_cost
            history.append((k, current_cost))
            improved = True
            # restart neighborhoods (VND typical behavior)
            k_index = 0
        else:
            k_index += 1
    return (current_cost == 0), iter_total, assignment, history

In [9]:
def run_experiment(k=3, n_values=None, m_values=None, instances_per_combo=10, seed=None):
    if n_values is None:
        n_values = [20]
    if m_values is None:
        m_values = [60]
    random_seed = seed or 12345
    random.seed(random_seed)

    results = []
    for n in n_values:
        for m in m_values:
            # accumulate stats per algorithm & heuristic
            stats = defaultdict(lambda: {"success":0, "runs":0, "time":0.0, "steps":0.0})
            for inst_idx in range(instances_per_combo):
                inst_seed = random.randint(0, 10**9)
                clauses = generate_k_sat(n, m, k, seed=inst_seed)
                # algorithms to run
                algos = [
                    ("HC", hill_climbing, {"max_iters":500}),
                    ("Beam3", beam_search, {"beam_width":3, "max_levels":200}),
                    ("Beam4", beam_search, {"beam_width":4, "max_levels":200}),
                    ("VND", vnd, {"max_no_improve_iter":500})
                ]
                heuristics = [("h1", h1), ("h2", h2)]
                for name, fn, kwargs in algos:
                    for hname, hfn in heuristics:
                        key = f"{name}_{hname}"
                        start_time = time.time()
                        if name.startswith("Beam"):
                            success, steps, assignment, history = fn(clauses, n, beam_width=kwargs["beam_width"], heuristic_fn=hfn, max_levels=kwargs["max_levels"])
                        elif name == "HC":
                            success, steps, assignment, history = fn(clauses, n, heuristic_fn=hfn, max_iters=kwargs["max_iters"], seed=inst_seed)
                        else: # VND
                            success, steps, assignment, history = fn(clauses, n, heuristic_fn=hfn, max_no_improve_iter=kwargs["max_no_improve_iter"], seed=inst_seed)
                        elapsed = time.time() - start_time
                        stats[key]["runs"] += 1
                        stats[key]["time"] += elapsed
                        stats[key]["steps"] += steps
                        if success:
                            stats[key]["success"] += 1
            # finalize averages
            for kkey, val in stats.items():
                val["avg_time"] = val["time"] / val["runs"]
                val["avg_steps"] = val["steps"] / val["runs"]
                val["penetrance"] = val["success"] / val["runs"]
            results.append(((n,m), stats))
            # print summary for this combo
            print(f"\n=== n={n}, m={m}, k={k} | instances={instances_per_combo} ===")
            print(f"{'Algo+Heur':<12} {'penetrance':>10} {'avg_time(s)':>12} {'avg_steps':>10}")
            for kkey, val in sorted(stats.items()):
                print(f"{kkey:<12} {val['penetrance']:10.2f} {val['avg_time']:12.3f} {val['avg_steps']:10.2f}")
    return results


In [10]:
if __name__ == "__main__":
    # small demo to show output quickly for inclusion in report
    # use small n, m and few instances
    demo_results = run_experiment(
        k=3,
        n_values=[8, 10],           # small sizes for quick demo
        m_values=[20, 30],
        instances_per_combo=6,
        seed=42
    )



=== n=8, m=20, k=3 | instances=6 ===
Algo+Heur    penetrance  avg_time(s)  avg_steps
Beam3_h1           1.00        0.000       0.50
Beam3_h2           1.00        0.000       1.50
Beam4_h1           1.00        0.000       0.83
Beam4_h2           1.00        0.000       1.00
HC_h1              0.67        0.000       1.33
HC_h2              0.67        0.000       1.33
VND_h1             1.00        0.002     124.00
VND_h2             1.00        0.002     124.00

=== n=8, m=30, k=3 | instances=6 ===
Algo+Heur    penetrance  avg_time(s)  avg_steps
Beam3_h1           1.00        0.001       1.67
Beam3_h2           1.00        0.001       1.83
Beam4_h1           1.00        0.001       1.67
Beam4_h2           1.00        0.002       2.00
HC_h1              0.33        0.001       1.83
HC_h2              0.33        0.000       1.83
VND_h1             0.83        0.004     126.00
VND_h2             0.83        0.007     126.00

=== n=10, m=20, k=3 | instances=6 ===
Algo+Heur    penetran